<a href="https://colab.research.google.com/github/AthulyaSK/Decision-variables-and-Constraints/blob/main/06Q3_AssignmentAnswers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set-Up

In [ ]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo. 
#Uncomment the appropriate solver that you need.
#for reference, see https://colab.research.google.com/drive/1yGk8RB5NXrcx9f1Tb-oCiWzbxh61hZLI?usp=sharing

#installing and importing pyomo
!pip install -q pyomo
from pyomo.environ import *

###installing and importing specific solvers (uncomment the one(s) you need)
###glpk
!apt-get install -y -qq glpk-utils
###cbc
#!apt-get install -y -qq coinor-cbc
###ipopt
#!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
#!unzip -o -q ipopt-linux64
###bonmin
#!wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
#!unzip -o -q bonmin-linux64
###couenne
#!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
#!unzip -o -q couenne-linux64
###geocode
#!wget -N -q "https://ampl.com/dl/open/gecode/gecode-linux64.zip"
#!unzip -o -q gecode-linux64

#Using the solvers:
#SolverFactory('glpk', executable='/usr/bin/glpsol')
#SolverFactory('cbc', executable='/usr/bin/cbc')
#SolverFactory('ipopt', executable='/content/ipopt')
#SolverFactory('bonmin', executable='/content/bonmin')
#SolverFactory('couenne', executable='/content/couenne')
#SolverFactory('gecode', executable='/content/gecode')

     |████████████████████████████████| 9.7 MB 5.2 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123991 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up 

In [ ]:
import pandas as pd

In [ ]:
#write code that solves one instance of the assignment problem
times = [[193,171,51],
         [189,128,35],
         [39,174,46]]
#first index is the person, second index is the task, times[i][j] denotes the time it takes person i to complete task j

#declare a concrete model
model = ConcreteModel()

num_people = 3 #use i to index the people, this is the first index
num_tasks = 3 #use j to index the tasks, this is the second index

#declare the decision variables
model.x = Var(range(num_people), range(num_tasks), domain=NonNegativeReals) #NOTE: This is how you define 2-dimensional decision variables, model.x[i,j]

#set the objective
model.Objective = Objective(expr = sum(model.x[i,j]*times[i][j] for i in range(num_people) for j in range(num_tasks)), sense = minimize) #this is how you sum up over multiple indices

#set the people constraints
model.PeopleConstraints = ConstraintList() #declaring an empty list of constraints
for i in range(num_people): #add one constraint per person
    model.PeopleConstraints.add(expr = sum(model.x[i,j] for j in range(num_tasks)) <= 1)
    
#set the task constraints
model.TaskConstraints = ConstraintList()
for j in range(num_tasks): #adding one constraint per task
    model.TaskConstraints.add(expr = sum(model.x[i,j] for i in range(num_people)) >= 1)

#model.pprint()


In [ ]:
#solve the model
opt = SolverFactory('glpk')
opt.solve(model, tee = False)

{'Problem': [{'Name': 'unknown', 'Lower bound': 218.0, 'Upper bound': 218.0, 'Number of objectives': 1, 'Number of constraints': 7, 'Number of variables': 10, 'Number of nonzeros': 19, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.020310163497924805}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [ ]:
#print solution
print("objective:", model.Objective())
for i in range(num_people):
    for j in range(num_tasks):
        if model.x[i,j]() > 0:
            print("person", i, "gets task", j)

objective: 218.0
person 0 gets task 2
person 1 gets task 1
person 2 gets task 0


In [ ]:
#alternative solution output
print("objective:", model.Objective())
assignment_matrix = [[model.x[i,j]() for j in range(num_tasks)] for i in range(num_people)]
assignment_matrix

objective: 218.0


[[0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [1.0, 0.0, 0.0]]

In [ ]:
#converting assignments to dataframe
pd.DataFrame(assignment_matrix, index = ["person1", "person2", "person3"], columns = ["task1","task2","task3"])

,task1,task2,task3
person1,0.0,0.0,1.0
person2,0.0,1.0,0.0
person3,1.0,0.0,0.0


In [ ]:
#load in data (located on the first sheet)
df2 = pd.read_excel('AssignmentProblemData.xlsx', sheet_name='Largeproblemdata') 
df2.head()

,PersonNum,Task1,Task2,Task3,Task4,Task5,Task6,Task7,Task8,Task9,...,Task38,Task39,Task40,Task41,Task42,Task43,Task44,Task45,Task46,Task47
0,1,328,240,99,244,310,83,209,76,66,...,330,134,182,84,341,193,327,54,160,312
1,2,153,111,96,308,297,114,51,178,167,...,114,59,336,91,308,104,253,199,62,118
2,3,82,246,163,277,68,55,82,232,313,...,86,113,95,151,160,185,96,320,232,254
3,4,149,171,81,172,163,298,319,100,155,...,101,337,222,161,192,207,190,292,53,133
4,5,201,211,224,323,133,112,165,186,116,...,144,195,307,253,280,237,232,277,204,130


# Solving 50 small (3x3) assignment problems
Here is [the excel data](https://uwmadison.box.com/shared/static/m21ihwgu4aqvfoz5s1ozx980p7zuk5qb.xlsx). 

In [ ]:
#load in data (located on the first sheet)
df = pd.read_excel('AssignmentProblemData.xlsx', sheet_name='50problemsdata') 
df.head()

,ProblemNum,Person1Task1,Person1Task2,Person1Task3,Person2Task1,Person2Task2,Person3Task3,Person3Task1,Person3Task2,Person3Task3.1
0,1,193,171,51,189,128,35,39,174,46
1,2,117,19,37,53,32,92,189,40,185
2,3,144,126,136,68,118,32,43,60,193
3,4,12,100,149,130,72,52,116,188,107
4,5,13,147,188,186,27,68,102,125,28


In [ ]:
def solvemodel(times):
  #write code that solves one instance of the assignment problem
  #first index is the person, second index is the task, times[i][j] denotes the time it takes person i to complete task j

  #declare a concrete model
  model = ConcreteModel()

  num_people = 3 #use i to index the people, this is the first index
  num_tasks = 3 #use j to index the tasks, this is the second index

  #declare the decision variables
  model.x = Var(range(num_people), range(num_tasks), domain=NonNegativeReals) #NOTE: This is how you define 2-dimensional decision variables, model.x[i,j]

  #set the objective
  model.Objective = Objective(expr = sum(model.x[i,j]*times[i][j] for i in range(num_people) for j in range(num_tasks)), sense = minimize) #this is how you sum up over multiple indices

  #set the people constraints
  model.PeopleConstraints = ConstraintList() #declaring an empty list of constraints
  for i in range(num_people): #add one constraint per person
      model.PeopleConstraints.add(expr = sum(model.x[i,j] for j in range(num_tasks)) <= 1)
      
  #set the task constraints
  model.TaskConstraints = ConstraintList()
  for j in range(num_tasks): #adding one constraint per task
      model.TaskConstraints.add(expr = sum(model.x[i,j] for i in range(num_people)) >= 1)

  #solve the model
  opt = SolverFactory('glpk')
  opt.solve(model, tee = False)

  #return relevant values
  assignment_matrix = [[model.x[i,j]() for j in range(num_tasks)] for i in range(num_people)]
  print (assignment_matrix)


In [ ]:
solutions = []
for i in range(len(df)):
  row = df.iloc[i].values.tolist()
  a = row[1:4]
  b = row[4:7]
  c = row[7:]
  times = []
  times.append(a)
  times.append(b)
  times.append(c)
  solutions.append(solvemodel(times))

print(solutions)

[[0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [1.0, 0.0, 0.0]]
[[0.0, 0.0, 1.0], [1.0, 0.0, 0.0], [0.0, 1.0, 0.0]]
[[0.0, 1.0, 0.0], [0.0, 0.0, 1.0], [1.0, 0.0, 0.0]]
[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]
[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]
[[0.0, 1.0, 0.0], [0.0, 0.0, 1.0], [1.0, 0.0, 0.0]]
[[0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [1.0, 0.0, 0.0]]
[[0.0, 0.0, 1.0], [1.0, 0.0, 0.0], [0.0, 1.0, 0.0]]
[[0.0, 1.0, 0.0], [1.0, 0.0, 0.0], [0.0, 0.0, 1.0]]
[[0.0, 0.0, 1.0], [1.0, 0.0, 0.0], [0.0, 1.0, 0.0]]
[[0.0, 1.0, 0.0], [1.0, 0.0, 0.0], [0.0, 0.0, 1.0]]
[[0.0, 1.0, 0.0], [0.0, 0.0, 1.0], [1.0, 0.0, 0.0]]
[[1.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0]]
[[0.0, 1.0, 0.0], [1.0, 0.0, 0.0], [0.0, 0.0, 1.0]]
[[0.0, 0.0, 1.0], [1.0, 0.0, 0.0], [0.0, 1.0, 0.0]]
[[0.0, 1.0, 0.0], [1.0, 0.0, 0.0], [0.0, 0.0, 1.0]]
[[0.0, 0.0, 1.0], [1.0, 0.0, 0.0], [0.0, 1.0, 0.0]]
[[0.0, 0.0, 1.0], [1.0, 0.0, 0.0], [0.0, 1.0, 0.0]]
[[0.0, 0.0, 1.0], [1.0, 0.0, 0.0], [0.0, 1.0, 0.0]]
[[0.0, 0.0, 